# Part10-11

In [1]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle

In [ ]:
def get_cur_hour_num_to(path,count_path='./data/train_hard.csv'):
    
    i = 0
    user_count = {}
    cur_hour_user_count = {}
    csv_reader = csv.reader(open(count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        user = row[1]
        hour = row[7]
        
        if user not in user_count:
            user_count[user] = 1
        else:
            user_count[user] += 1
        if hour not in cur_hour_user_count:
            cur_hour_user_count[hour] = {}
            cur_hour_user_count[hour][user] = 1
        else:
            if user not in cur_hour_user_count[hour]:
                cur_hour_user_count[hour][user] = 1
            else:
                cur_hour_user_count[hour][user] += 1
    
    

    cur_hour_user_end_dic = {}# 记录当前小时内用户使用了哪些终点几次
    
    cur_hour_end_node_user_dic = {}# 记录当前小时内那些用户使用了终点

    
    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        end = row[6]
        hour = row[7]
        
        
        # 记录当前小时那些用户使用了哪些终点几次
        if hour not in cur_hour_user_end_dic:
            cur_hour_user_end_dic[hour] = {}
            cur_hour_user_end_dic[hour][user] = {}
            cur_hour_user_end_dic[hour][user][end] = 1
        else:
            if user not in cur_hour_user_end_dic[hour]:
                cur_hour_user_end_dic[hour][user] = {}
                cur_hour_user_end_dic[hour][user][end] = 1
            else:
                if end not in cur_hour_user_end_dic[hour][user]:
                    cur_hour_user_end_dic[hour][user][end] = 1
                else:
                    cur_hour_user_end_dic[hour][user][end] += 1
        
        
        # 记录当前小时那些用户在那个终点到达过  # 记录当前小时哪些终点被哪些用户使用过
        if hour not in cur_hour_end_node_user_dic:
            cur_hour_end_node_user_dic[hour] = {}
            cur_hour_end_node_user_dic[hour][end] = set()
            cur_hour_end_node_user_dic[hour][end].add(user)
        else:
            if end not in cur_hour_end_node_user_dic[hour]:
                cur_hour_end_node_user_dic[hour][end] = set()
                cur_hour_end_node_user_dic[hour][end].add(user)
            else:
                cur_hour_end_node_user_dic[hour][end].add(user)

    
    num_user_cur_hour_to_ratio = {}# 当前小时去过此终点的用户去该终点的总数/当前小时去过此终点的用户使用的总次数
    num_user_cur_hour_to_cur_hour_ratio = {}# 当前小时去过此终点的用户去该终点的总数/当前小时去过此终点的用户当前小时使用的总次数
    
    
    for hour in cur_hour_end_node_user_dic:
        
        
        num_user_cur_hour_to_ratio[hour] = {}
        num_user_cur_hour_to_cur_hour_ratio[hour] = {}
        
        
        for node in cur_hour_end_node_user_dic[hour]:# 当前小时内被到达过的点


            total_to = 0# 当前小时内该点被到达几次==当前小时内去过该点的所有用户一共去过几次
            total_used = 0# 当前小时去过该点的用户骑行总次数
            total_cur_hour_used = 0# 当前小时去过该点的用户当前小时骑行总次数

            for user in cur_hour_end_node_user_dic[hour][node]:# 遍历当前小时内去过node的用户
                
                
                #if hour in cur_hour_user_count and \
                        #user in cur_hour_user_end_dic[hour] \
                            #and node in cur_hour_user_end_dic[hour][user]:
                total_to+=cur_hour_user_end_dic[hour][user][node]# 累计当前小时到达过这个点的用户当前小时到达这个点几次
                
                total_used+=user_count[user]# 当前小时去过该点的用户总使用次数
                
                #if hour in cur_hour_user_count and user in cur_hour_user_count[hour]:
                total_cur_hour_used += cur_hour_user_count[hour][user]# 当前小时去过该点的用户当前小时骑行总次数


            if total_used!=0:
                num_user_cur_hour_to_ratio[hour][node] = total_to/total_used
            else:
                print('xxxx')
                num_user_cur_hour_to_ratio[node] = 0
            
            if total_cur_hour_used!=0:
                num_user_cur_hour_to_cur_hour_ratio[hour][node] = total_to/total_cur_hour_used
            else:
                print('xxxx')
                num_user_cur_hour_to_cur_hour_ratio[node] = 0
            
    return num_user_cur_hour_to_ratio,num_user_cur_hour_to_cur_hour_ratio

In [ ]:
num_user_cur_hour_to_ratio,num_user_cur_hour_to_cur_hour_ratio = get_cur_hour_num_to('./data/train_hard.csv')

In [ ]:
with open('./data/temp/feature_table/num_user_cur_hour_to_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_to_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_cur_hour_to_cur_hour_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_to_cur_hour_ratio, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
num_user_cur_hour_to_geofly_ratio,num_user_cur_hour_to_geofly_cur_hour_ratio\
                    = get_cur_hour_num_to('./data/train_end_geofly.csv')

In [ ]:
with open('./data/temp/feature_table/num_user_cur_hour_to_geofly_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_to_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_cur_hour_to_geofly_cur_hour_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_to_geofly_cur_hour_ratio, f, pickle.HIGHEST_PROTOCOL)

In [2]:
def get_cur_hour_num_from(path,count_path='./data/train_hard.csv'):
    
    i = 0
    user_count = {}
    cur_hour_user_count = {}
    csv_reader = csv.reader(open(count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        user = row[1]
        hour = row[7]
        
        if user not in user_count:
            user_count[user] = 1
        else:
            user_count[user] += 1
        if hour not in cur_hour_user_count:
            cur_hour_user_count[hour] = {}
            cur_hour_user_count[hour][user] = 1
        else:
            if user not in cur_hour_user_count[hour]:
                cur_hour_user_count[hour][user] = 1
            else:
                cur_hour_user_count[hour][user] += 1
    
    

    cur_hour_user_start_dic = {}# 记录当前小时内用户使用了哪些起点几次
    
    cur_hour_start_node_user_dic = {}# 记录当前小时内那些用户使用了起点

    
    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        hour = row[7]
        
        
        # 记录当前小时那些用户使用了哪些终点几次
        if hour not in cur_hour_user_start_dic:
            cur_hour_user_start_dic[hour] = {}
            cur_hour_user_start_dic[hour][user] = {}
            cur_hour_user_start_dic[hour][user][start] = 1
        else:
            if user not in cur_hour_user_start_dic[hour]:
                cur_hour_user_start_dic[hour][user] = {}
                cur_hour_user_start_dic[hour][user][start] = 1
            else:
                if start not in cur_hour_user_start_dic[hour][user]:
                    cur_hour_user_start_dic[hour][user][start] = 1
                else:
                    cur_hour_user_start_dic[hour][user][start] += 1
        
        
        # 记录当前小时那些用户在那个终点到达过  # 记录当前小时哪些终点被哪些用户使用过
        if hour not in cur_hour_start_node_user_dic:
            cur_hour_start_node_user_dic[hour] = {}
            cur_hour_start_node_user_dic[hour][start] = set()
            cur_hour_start_node_user_dic[hour][start].add(user)
        else:
            if start not in cur_hour_start_node_user_dic[hour]:
                cur_hour_start_node_user_dic[hour][start] = set()
                cur_hour_start_node_user_dic[hour][start].add(user)
            else:
                cur_hour_start_node_user_dic[hour][start].add(user)

    
    num_user_cur_hour_from_ratio = {}# 当前小时去过此终点的用户去该终点的总数/当前小时去过此终点的用户使用的总次数
    num_user_cur_hour_from_cur_hour_ratio = {}# 当前小时去过此终点的用户去该终点的总数/当前小时去过此终点的用户当前小时使用的总次数
    
    
    for hour in cur_hour_start_node_user_dic:
        
        
        num_user_cur_hour_from_ratio[hour] = {}
        num_user_cur_hour_from_cur_hour_ratio[hour] = {}
        
        
        for node in cur_hour_start_node_user_dic[hour]:# 当前小时内被到达过的点


            total_to = 0# 当前小时内该点被到达几次==当前小时内去过该点的所有用户一共去过几次
            total_used = 0# 当前小时去过该点的用户骑行总次数
            total_cur_hour_used = 0# 当前小时去过该点的用户当前小时骑行总次数

            for user in cur_hour_start_node_user_dic[hour][node]:# 遍历当前小时内去过node的用户
                
                
                #if hour in cur_hour_user_count and \
                        #user in cur_hour_user_end_dic[hour] \
                            #and node in cur_hour_user_end_dic[hour][user]:
                total_to+=cur_hour_user_start_dic[hour][user][node]# 累计当前小时到达过这个点的用户当前小时到达这个点几次
                
                total_used+=user_count[user]# 当前小时去过该点的用户总使用次数
                
                #if hour in cur_hour_user_count and user in cur_hour_user_count[hour]:
                total_cur_hour_used += cur_hour_user_count[hour][user]# 当前小时去过该点的用户当前小时骑行总次数


            if total_used!=0:
                num_user_cur_hour_from_ratio[hour][node] = total_to/total_used
            else:
                print('xxxx')
                num_user_cur_hour_from_ratio[node] = 0
            
            if total_cur_hour_used!=0:
                num_user_cur_hour_from_cur_hour_ratio[hour][node] = total_to/total_cur_hour_used
            else:
                print('xxxx')
                num_user_cur_hour_from_cur_hour_ratio[node] = 0
            
    return num_user_cur_hour_from_ratio,num_user_cur_hour_from_cur_hour_ratio

In [3]:
num_user_cur_hour_from_ratio,num_user_cur_hour_from_cur_hour_ratio= get_cur_hour_num_from('./data/train_hard.csv')

In [20]:
with open('./data/temp/feature_table/num_user_cur_hour_from_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_from_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_cur_hour_from_cur_hour_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_from_cur_hour_ratio, f, pickle.HIGHEST_PROTOCOL)

In [21]:
num_user_cur_hour_from_geofly_ratio,num_user_cur_hour_from_geofly_cur_hour_ratio \
                    = get_cur_hour_num_from('./data/train_start_geofly.csv')

In [41]:
with open('./data/temp/feature_table/num_user_cur_hour_from_geofly_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_from_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_cur_hour_from_geofly_cur_hour_ratio','wb+') as f:
    pickle.dump(num_user_cur_hour_from_geofly_cur_hour_ratio, f, pickle.HIGHEST_PROTOCOL)